In [1]:
import numpy as np
import xgboost as xgb

In [2]:
X = np.load("./tatanic_X_train.npy")
y = np.load("./tatanic_y_train.npy")

In [3]:
from sklearn.cross_validation import train_test_split

/home/sc82choi/miniconda3/envs/dsme/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [5]:
from xgboost import XGBClassifier

model = XGBClassifier(n_estimators=1000, max_depth=2, learning_rate=0.5, nthread=7)
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.5,
       max_delta_step=0, max_depth=2, min_child_weight=1, missing=None,
       n_estimators=1000, n_jobs=1, nthread=7, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=None, subsample=1, verbosity=1)

In [6]:
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.5,
       max_delta_step=0, max_depth=2, min_child_weight=1, missing=None,
       n_estimators=1000, n_jobs=1, nthread=7, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=None, subsample=1, verbosity=1)

In [7]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

/home/sc82choi/miniconda3/envs/dsme/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [8]:
sum(y_test == predictions) / (len(y_test)*1.0)

0.8202247191011236

In [9]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [10]:
param = {'max_depth': 2, 'eta': 0.5, 'silent': 1, 'objective': 'binary:logistic'}
param['nthread'] = 7
param['eval_metric'] = 'auc'
evallist = [(dtest, 'eval'), (dtrain, 'train')]
plst = param.items()

In [11]:
num_round = 50
bst = xgb.train(plst, dtrain, num_round, evallist)

[0]	eval-auc:0.792066	train-auc:0.849547
[1]	eval-auc:0.824281	train-auc:0.871341
[2]	eval-auc:0.829192	train-auc:0.873058
[3]	eval-auc:0.827036	train-auc:0.879254
[4]	eval-auc:0.824072	train-auc:0.888967
[5]	eval-auc:0.823383	train-auc:0.89309
[6]	eval-auc:0.830329	train-auc:0.896471
[7]	eval-auc:0.829491	train-auc:0.898675
[8]	eval-auc:0.836737	train-auc:0.900949
[9]	eval-auc:0.836707	train-auc:0.902318
[10]	eval-auc:0.839641	train-auc:0.905666
[11]	eval-auc:0.842904	train-auc:0.908055
[12]	eval-auc:0.841317	train-auc:0.910171
[13]	eval-auc:0.838353	train-auc:0.910586
[14]	eval-auc:0.842156	train-auc:0.911993
[15]	eval-auc:0.839521	train-auc:0.913743
[16]	eval-auc:0.836886	train-auc:0.91442
[17]	eval-auc:0.835329	train-auc:0.915107
[18]	eval-auc:0.837695	train-auc:0.915412
[19]	eval-auc:0.833713	train-auc:0.916825
[20]	eval-auc:0.833293	train-auc:0.918161
[21]	eval-auc:0.832874	train-auc:0.919044
[22]	eval-auc:0.832994	train-auc:0.920282
[23]	eval-auc:0.835659	train-auc:0.923271
[24]

In [12]:
bst.save_model('0001.model')


In [13]:
# bst = xgb.Booster({'nthread': 4})  # init model
# bst.load_model('0001.model')  # load data


In [14]:
ypred = bst.predict(dtest, ntree_limit=bst.best_ntree_limit)


In [15]:
(sum((ypred>0.5) == y_test)) / 267.0

0.7940074906367042

In [16]:
bst = xgb.Booster({'nthread': 4})  # init model
bst.load_model('model.bin')  # load data


XGBoostError: [08:36:53] /workspace/dmlc-core/src/io/local_filesys.cc:209: Check failed: allow_null:  LocalFileSystem::Open "model.bin": No such file or directory
Stack trace:
  [bt] (0) /home/sc82choi/miniconda3/envs/dsme/xgboost/libxgboost.so(dmlc::io::LocalFileSystem::Open(dmlc::io::URI const&, char const*, bool)+0x4d8) [0x7fbb94d57fc8]
  [bt] (1) /home/sc82choi/miniconda3/envs/dsme/xgboost/libxgboost.so(dmlc::Stream::Create(char const*, char const*, bool)+0x39) [0x7fbb94d2f369]
  [bt] (2) /home/sc82choi/miniconda3/envs/dsme/xgboost/libxgboost.so(XGBoosterLoadModel+0x27) [0x7fbb94aa0407]
  [bt] (3) /home/sc82choi/miniconda3/envs/dsme/lib/python3.6/lib-dynload/../../libffi.so.6(ffi_call_unix64+0x4c) [0x7fbbff07dec0]
  [bt] (4) /home/sc82choi/miniconda3/envs/dsme/lib/python3.6/lib-dynload/../../libffi.so.6(ffi_call+0x22d) [0x7fbbff07d87d]
  [bt] (5) /home/sc82choi/miniconda3/envs/dsme/lib/python3.6/lib-dynload/_ctypes.cpython-36m-x86_64-linux-gnu.so(_ctypes_callproc+0x2ce) [0x7fbbff29

In [17]:
xgb.to_graphviz(bst, num_trees=2)


ImportError: You must install graphviz to plot tree